In [1]:
from torch.utils.data import Dataset
#import json
#print(Dataset)
class gpt2train(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = open(path)
        self.Y=[]

        self.X = []

        for i in self.data:
            self.Y.append(i.strip())

        #for idx,i in enumerate(self.Y):
         #       self.X[idx] = "<startofstring> "+i+" <tag>: "+self.X[idx+1]+" <endofstring>"
        for i in range(0,len(self.Y),2):
            #print(self.Y[i],self.Y[i+1],i,len(self.Y))
            self.X.append( "<startofstring> "+self.Y[i]+" <tag>: "+(self.Y[i+1]).split(':')[1]+" <endofstring>")

        #self.X = self.X[:500]

        #print(self.X)

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [2]:
!pip install transformers
!pip install gpt2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#****** test and train set generation*****
import json
import pandas as pd
f=open("ssss.json")
da=json.load(f)
l=[]
for i in da:
    for j in da[i]:
        for k in da[i][j]:
            if(k[0].upper()!=k[0].lower()):#to remove texts that are just numbers or special characters
                l.append(k)
emp=pd.DataFrame(data=l)
#print(emp)

s1=set((",".join(set(emp[2]))).split(","))
#print(s1)
dict={}
p=["homicide_murder","life_imprisonment","riot","physical_assault","evidence_inconsistency","witness_testimony","expert_witness_testimony"]
s=[i for i in s1 if i not in p] #list of all other tags
#print(s)
for i in p:
    dict[i]=list(emp[emp[2].str.contains(i)][0])
'''
for i in s: #loop to generate others tag from tags unused
    dict["others"]=list(emp[emp[2].str.contains(i)][0])
'''
#print(dict)

prompt=""
g=open("promptfile.txt","w+")
for i in dict:
    r=min(20,len(dict[i]))
    j=0
    c=0
    while c<r:
        if dict[i][j] not in prompt:
            c=c+1
            m=emp[emp[0]==dict[i][j]].index.values
            for k in m:
                emp=emp.drop(k)
            prompt+="Statement:"+dict[i][j]+". \n"
            g.write("Statement:"+dict[i][j]+". \n")
            prompt+="Tag:"+i+"\n"
            g.write("Tag:"+i+"\n")
        j=j+1

h=open("ssss2.txt","r+")
for i in (h.read()).split(". ")[:10]:
    prompt+="Statement:"+i+". \n"
    g.write("Statement:"+i+". \n")
    prompt+="Tag:"+"Others"+"\n"
    g.write("Tag:"+"Others"+"\n")
h.close
g.close()


#**********test and train set generation end***********

In [1]:
#!pip install torch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00


In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
#from gpt2train import gpt2train
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

def train(chatData, model, optim):

    epochs = 65

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            #print(X,a)
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("He was charged under section 302"))

def infer(inp):
    inp = "<startofstring> "+inp+" <tag>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a, no_repeat_ngram_size=2,early_stopping=True,max_new_tokens=10)
    #output=model.generate(inp,no_repeat_ngram_size=2,early_stopping=True,max_new_tokens=3)
    output = tokenizer.decode(output[0])
    return output




device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>",
                                "sep_token": "<tag>:"})
#tokenizer.add_tokens(["<tag>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = gpt2train("./promptfile_edited.txt", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)
print(chatData)
model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

print("infer from model : ")
'''
while True:
  inp = input()
  print(infer(inp))
'''


training .... 


  2%|▏         | 1/65 [00:03<03:34,  3.35s/it]

<startofstring> He was charged under section 302 <tag>: <startofstring> <startofstring> <endofstring> <startofstring> <pad> <startofstring> <tag>: <pad> <endofstring> <endofstring>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  3%|▎         | 2/65 [00:06<03:32,  3.37s/it]

<startofstring> He was charged under section 302 <tag>:  the.... the, the the....


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 98%|█████████▊| 64/65 [05:44<00:04,  4.68s/it]

<startofstring> He was charged under section 302 <tag>:  302. <tag>: expert_witness_test


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 65/65 [05:48<00:00,  5.36s/it]

<startofstring> He was charged under section 302 <tag>: homicide_murder <endofstring> <pad> <pad> <startofstring> <pad>
infer from model : 


'\nwhile True:\n  inp = input()\n  print(infer(inp))\n'

In [12]:
#*****file creation for testing******

#print("prompt: ",prompt)
#print(emp)
pf=open('promptfile_edited.txt')
l=''
for i in pf:
  s=i.split(':')
  if "Statement" in s[0]:
    l+=(s[1])
dh={}
dh["Statement"]=[]
dh["Assigned tag"]=[]
p=["homicide_murder","life_imprisonment","riot","physical_assault","evidence_inconsistency","witness_testimony","expert_witness_testimony"]
for j in p:
    dl=list(emp[emp[2]==j].index.values)
    c=0
    for i in dl:
        if emp[0][i] not in dh["Statement"] and emp[0][i] not in l:
            dh["Statement"].append(emp[0][i])
            dh["Assigned tag"].append(emp[2][i])
            c=c+1
        if c==20:
            break
    #print(dh)
'''
for j in s: #to generate statements from others tag in the test set
    c=0
    for i in emp.index.values:
        if j in emp[2][i]:
           dh["Statement"].append(emp[0][i])
           dh["Assigned tag"].append(emp[2][i])
           c+=1
        if c==2:
            break
'''

p.append("others")

hm=pd.DataFrame(data=dh)
print(hm)
#hm.to_csv('h2_test.csv', index=False)

ip=list(hm.index.values)
#print(ip)
#r=1005 # The starting index from which checking should be started
l=[]
for i in ip:
    print("Statement:",hm["Statement"][i])
    s = infer(hm["Statement"][i])+'<endofstring>'
    print(s)
    j=s.index('<endofstring>')
    m=s.index('<tag>:')
    #p=s[i:].split(':')[1]
    response=s[m:j].split(':')[1]
    #print(p)
    print("Generated Tag:",response)
    print("Assigned Tag:",hm["Assigned tag"][i])
    l.append(response)
hm["generated response"]=l
hm.to_excel('gpt2_test.xlsx', index=False)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


                                             Statement  \
0    The High Court also held that the case of the ...   
1            conviction and sentence under Section 302   
2    charge under Section 302 read with Section 34 ...   
3    three sons Kulwant Singh, Gurmit Singh and Man...   
4    appeal is held devoid of any merit and is dism...   
..                                                 ...   
120  An incised wound 8 cms x 3 cms x 2 cms over th...   
121  Laceration 8 cms x 4 cms x 2 cm over the inner...   
122  Laceration 4 x 3 x 1 cm over inner side of lef...   
123  Laceration 5 cms x cms x 1 cm over the dorsal ...   
124                            Heart : Chambers empty.   

                 Assigned tag  
0             homicide_murder  
1             homicide_murder  
2             homicide_murder  
3             homicide_murder  
4             homicide_murder  
..                        ...  
120  expert_witness_testimony  
121  expert_witness_testimony  
122  expert_wit

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> conviction and sentence under Section 302 <tag>: homicide_murder <endofstring> <pad> <pad> <startofstring> <pad><endofstring>
Generated Tag:  homicide_murder 
Assigned Tag: homicide_murder
Statement: charge under Section 302 read with Section 34 of the IPC and convicted all of them under the aforesaid sections.
<startofstring> charge under Section 302 read with Section 34 of the IPC and convicted all of them under the aforesaid sections. <tag>: homicide_murder <endofstring> <pad> <pad> <startofstring> <pad><endofstring>
Generated Tag:  homicide_murder 
Assigned Tag: homicide_murder
Statement: three sons Kulwant Singh, Gurmit Singh and Mangal Singh were tried on charges under Sections 302


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

<startofstring> A stab wound 3 cms x 2 cms x 6 cms over the centre of back 7 cms below injury no.14. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony
Statement: A stab wound 3 cms x 2 cms x 6 cms over the back of the right side of the chest 6 cms away from injury no.14.
<startofstring> A stab wound 3 cms x 2 cms x 6 cms over the back of the right side of the chest 6 cms away from injury no.14. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony
Statement: An incised wound 8 cms x 3 cms x 2 cms over the back of the right side of the chest 6 cms away from injury no.15.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> An incised wound 8 cms x 3 cms x 2 cms over the back of the right side of the chest 6 cms away from injury no.15. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony
Statement: Laceration 8 cms x 4 cms x 2 cm over the inner side of the left elbow.
<startofstring> Laceration 8 cms x 4 cms x 2 cm over the inner side of the left elbow. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony
Statement: Laceration 4 x 3 x 1 cm over inner side of left palm.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> Laceration 4 x 3 x 1 cm over inner side of left palm. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony
Statement: Laceration 5 cms x cms x 1 cm over the dorsal aspect of the left palm.
<startofstring> Laceration 5 cms x cms x 1 cm over the dorsal aspect of the left palm. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony
Statement: Heart : Chambers empty.
<startofstring> Heart : Chambers empty. <tag>: expert_witness_testimony <endofstring> <pad><endofstring>
Generated Tag:  expert_witness_testimony 
Assigned Tag: expert_witness_testimony


In [5]:
pf=open('promptfile_edited.txt')
l=''
for i in pf:
  s=i.split(':')
  if "Statement" in s[0]:
    l+=(s[1])
print(l)

The Sessions Judge framed charge under Section 302/34 of the IPC. 
we do not find any cogent reason to interfere with the judgment of the High Court converting the offence to Section 304 Part II of the IPC from Section 302 of the IPC. 
having been charged of the offences under Sections 302 and 397 IPC for having committed the murder of a young girl. 
charged for offences punishable under Sections 302. 
And thereby committed offences punishable under Section 3(3) of TADA (P) Act, 1987 and Section 120-B of IPC read with Sections 3(2)(i)(ii), 3(3)(4), 5 and 6 of TADA (P) Act, 1987 and read with Sections 302. 
The appellant, by causing the death of 3 persons as mentioned above, committed an offence punishable under Section 302. 
convicting them under Section 302. 
High Court by convicting them under Section 302. 
appellants under the impugned order stood convicted for an offence punishable under Sections 148 and 302. 
appellants along with Bhagirath (A-5) and Bali Ram (A-7) were tried for 

In [14]:
import pandas as pd
fd=pd.read_csv("gpt2_test_data_edited.csv")
print(fd)
p=["homicide_murder","life_imprisonment","riot","physical_assault","evidence_inconsistency","witness_testimony","expert_witness_testimony"]
dd={}
dd1={}
for i in p:
    tp=0
    fp=0
    fn=0
    tn=0
    for (m,k) in zip(fd["Assigned tag"],fd["generated response"]):
        for j in (m.strip()).split(","):
            if i==j and i==k.strip():
                tp+=1
            elif i!=j and i==k.strip():
                fp+=1
            elif i==j and i!=k.strip():
                fn+=1
            elif i!=j and i!=k.strip():
                tn+=1
    dd[i]=[tp,fp,fn,tn]

'''
tp,fp,fn,tn=0,0,0,0
for (i,j) in zip(fd["Assigned tag"],fd["generated response"]):
    if i not in p:
        if "others" in j:
            tp+=1
        else:
            fn+=1
    else:
        if "others" in j:
            fp+=1
        else:
            tn+=1
dd["others"]=[tp,fp,fn,tn]
'''

print(dd)
#p.append("others")
dd1["labels"]=p
dd1["Accuracy"]=[(dd[i][0]+dd[i][3])/(dd[i][0]+dd[i][1]+dd[i][2]+dd[i][3]) for i in dd.keys()]
dd1["Misclassification"]=[(dd[i][1]+dd[i][2])/(dd[i][0]+dd[i][1]+dd[i][2]+dd[i][3]) for i in dd.keys()]
dd1["Precision"]=[dd[i][0]/(dd[i][0]+dd[i][1]) for i in dd.keys()]
dd1["Sensitivity"]=[dd[i][0]/(dd[i][0]+dd[i][2]) for i in dd.keys()]
dd1["Specificity"]=[dd[i][3]/(dd[i][3]+dd[i][1]) for i in dd.keys()]
em=pd.DataFrame(data=dd1)
print(em)
em.to_csv('data_eval_edited.csv', index=False)

                                             Statement  \
0    The High Court also held that the case of the ...   
1            conviction and sentence under Section 302   
2    charge under Section 302 read with Section 34 ...   
3    three sons Kulwant Singh, Gurmit Singh and Man...   
4    appeal is held devoid of any merit and is dism...   
..                                                 ...   
119  An incised wound 8 cms x 3 cms x 2 cms over th...   
120  Laceration 8 cms x 4 cms x 2 cm over the inner...   
121  Laceration 4 x 3 x 1 cm over inner side of lef...   
122  Laceration 5 cms x cms x 1 cm over the dorsal ...   
123                            Heart : Chambers empty.   

                 Assigned tag          generated response  
0             homicide_murder            homicide_murder   
1             homicide_murder            homicide_murder   
2             homicide_murder            homicide_murder   
3             homicide_murder            homicide_murder   
4  